In [124]:
import pickle
import pandas as pd
import datetime
import numpy as np


In [175]:
with open('./submission_data/4_ts_prediction/all_preds', 'rb') as f:
    array = pickle.load(f)
    
with open('./submission_data/4_ts_prediction/mse_after', 'rb') as f:
    mse_after = pickle.load(f)

with open('./submission_data/4_ts_prediction/mse_before', 'rb') as f:
    mse_before = pickle.load(f)

In [176]:
sample_sub_2 = pd.read_csv('./sample_submissions/sample_sub_4.csv')
sample_sub_2['date'] = pd.to_datetime(sample_sub_2['date'])
# sample_sub_2.drop(['delta_stage_max', 'year', 'day', 'month'], axis=1, inplace=True)
sample_sub_2

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0
1,1993,3019,4,112,1993-04-22,0
2,1993,3019,4,113,1993-04-23,0
3,1993,3019,4,114,1993-04-24,0
4,1993,3019,4,115,1993-04-25,0
...,...,...,...,...,...,...
2480,2013,3230,6,162,2013-06-11,0
2481,2013,3230,6,163,2013-06-12,0
2482,2013,3230,6,164,2013-06-13,0
2483,2013,3230,6,165,2013-06-14,0


In [177]:
def create_submission(preds, mse, mse_tune):
    df = pd.DataFrame(columns=['year', 'station_id', 'month', 'day', 'date', 'delta_stage_max'])
    
    for index, row in enumerate(preds):
        new_row = {}
        date = pd.to_datetime(row[0])
        station_id = int(row[1])
        
        new_row['station_id'] = station_id
        new_row['year'] = date.year
        new_row['month'] = date.month
        new_row['day'] = date.timetuple().tm_yday
        
        if mse_tune[index] > mse[index]:
            predictions = row[3]
        else:
            predictions = row[2]
            
        for i, val in enumerate(predictions):
            if i == 0:
                new_row['delta_stage_max'] = predictions[i + 1] - predictions[i]
            else:
                new_row['delta_stage_max'] = predictions[i] - predictions[i - 1]
            
            new_row['stage_max_before'] = row[2][i]
            new_row['stage_max_after'] = row[3][i]
            new_row['stage_max'] = val
            new_row['date'] = date + datetime.timedelta(days=i)
            new_row['day'] = (date + datetime.timedelta(days=i)).timetuple().tm_yday
            new_row['mse_after'] = mse[index]
            new_row['mse_before'] = mse_tune[index]
            new_row['mse_diff'] = mse_tune[index] - mse[index]
            
            df = df.append(new_row, ignore_index=True)
            
    
            
    return df
            

In [178]:
results = create_submission(array, mse_after, mse_before)
results = results.iloc[:,:-6]
results.head()

,year,station_id,month,day,date,delta_stage_max
0,1993,3230,4,111,1993-04-21,2.406849
1,1993,3230,4,112,1993-04-22,2.406849
2,1993,3230,4,113,1993-04-23,2.904224
3,1993,3230,4,114,1993-04-24,3.510361
4,1993,3230,4,115,1993-04-25,3.647050


In [179]:
delta_stage_max = []

for index, row in sample_sub_2.iterrows():
    station_id = row['station_id']
    date = row['date']
    delta_stage_max.append(np.mean(results[(results['date'] == date) & (results['station_id'] == station_id)]['delta_stage_max']))
    
sample_sub_2['delta_stage_max'] = delta_stage_max
sample_sub_2.to_csv('submission_4.csv', index=False)

In [167]:
results = sample_sub_2.merge(results, how="inner", on=["station_id", "date"])
results = results.reindex(columns=['year', 'station_id', 'month', 'day', 'date', 'delta_stage_max'])
results.head()

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,1.415671
1,1993,3019,4,112,1993-04-22,1.415671
2,1993,3019,4,113,1993-04-23,1.358971
3,1993,3019,4,114,1993-04-24,1.190436
4,1993,3019,4,115,1993-04-25,1.592192


In [168]:
results.to_csv('submission_3.csv', index=False)

In [148]:
sample_sub_1['delta_stage_max'] = pd.read_csv('submission_2.csv')['delta_stage_max']
sample_sub_1.to_csv('submission_2.csv', index=False)